# Power system security Analysis

### Importing Libraries

Note you will need Numpy, Pandas, sklearn, matplotlib, scipy libraries to run this code so make sure they are installed

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.linear_model import LogisticRegressionCV as LGR
from sklearn.linear_model import LinearRegression as LNR
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.svm import SVC as SVM
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from scipy import stats
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
%matplotlib inline

#### Setting up working path

In [ ]:
''' Setting the Diirectory path'''
dir_path = (os.getcwd() + "\\").replace("\\","/") # If it does not work change it to path where data is stored.
print("Working directory is : ", dir_path)

### Importing Data Files

In [ ]:
data_all = pd.read_excel(dir_path + 'HW_TESLA.xls')
data_all = data_all.fillna(0)
data_all.head()

### Visualizing Correlation among data features

In [ ]:
plt.figure(figsize=[7,7])
plt.matshow(data_all.corr(), fignum=1)
plt.colorbar()
plt.show()

Data is highly correlated so We can reduce number of features to look at by using feature selction methods.

### Splitting Training and Test data

Splitting of data has been done considering that the data domain is uniform for train and test data.

In [ ]:
data_arr = data_all.values
Y = data_arr[:,0]
X = data_arr[:,1:]
X_train, X_tt, Y_train, Y_tt = train_test_split(X, Y,stratify=Y,test_size=0.5, random_state = 15)
X_val, X_test, Y_val, Y_test = train_test_split(X_tt, Y_tt,stratify=Y_tt,test_size=0.5, random_state =30)
print('Training Data Size', X_train.shape)

### Feature Selection

Feature selection can significantly reduce the number of observations that are needed to be taken for a given task. We are using Select K best feature selector and chi sqaure testing which selects the features which has 5 best chi square values according to the classes.

In [ ]:
sel = SelectKBest(chi2, k=5)
sel.fit(X_train - np.min(np.min(X_train)) + 20,Y_train)
X_train = sel.transform(X_train)
X_train.shape

In [ ]:
indices = sel.get_support(True)
indices = np.concatenate((np.array([-1]),indices))
indices

In [ ]:
data_all.columns[indices+1]

In [ ]:
new_data = data_all[data_all.columns[indices+1]]
new_data.head()

In [ ]:
X_test = sel.transform(X_test)
X_val = sel.transform(X_val)

The final features that are needed to be observed for classification are 'RINALDI', 'TOLUCA', 'MIGUEL', 'MIGUELMP', 'VALLEYSC' values and are sufficient for classification as we will see down.

### Model class for one part of ensemmble methods

Many Classification models can over fit and gives complex decision boundariesover data but assuming that every model overfits the data in a different waywe have applied ensemble method over all the models by taking the majorityvoting of models (arbiter over all models).  This approach guarantees to reducethe over fitting of individual models.

In [ ]:
class emodel:
    '''
    This is a class for a individual model in an ensemble model.
    '''
    # Constructor
    def __init__(self, name, **kwargs):
        self.em = None
        self.name = name
        # Check for type of model
        '''
        Only following models allowed you can check the meaning in import statements
        '''
        if(self.name == 'RFC'):
            self.em = RFC(**kwargs)
        elif(self.name == 'LGR'):
            self.em = LGR(**kwargs)
        elif(self.name == 'LNR'):
            self.em = LNR(**kwargs)
        elif(self.name == 'SVM'):
            self.em = SVM(**kwargs)
        elif(self.name == 'MLP'):
            self.em = MLP(**kwargs)
        elif(self.name == 'GNB'):
            self.em = GNB(**kwargs)
        elif(self.name == 'RNC'):
            self.em = RNC(**kwargs)
        elif(self.name == 'KNC'):
            self.em = KNC(**kwargs)
        else:
            pass
    def fit(self,X,Y):
        '''
        Training this model
        '''
        self.em.fit(X,Y)
    def predict(self,X):
        '''
        predicting using this model
        '''
        return self.em.predict(X)

### Ensemmble Model Class

This Class ensembles all the individual models

In [ ]:
class classifier:
    '''
    Ensemble Model
    
    What does it do?
    -------------------
    Applies PCA on data
    Normalizes data
    Trains individual models
    Predicts the classes and score
    '''
    # Constructor
    def __init__(self):
        self.models = [] #List of individual models
        self.fr = None
      
    # setting presets mean and standard deviation to normalize each test point
    def normalize_set(self,X): 
        '''
        Setting the training data mean and std to normalize test data too
        '''
        self.mu = X.mean(axis=0)
        self.std = X.std(axis=0)
        for i in range(len(self.std)): # removing any zero valued standard deviation.
            if(self.std[i] == 0):
                self.std[i] = 1e-60 #If std is zero make is near to 0 to avoid 1/0 error
    
    # normalizing X with presets
    def normalize(self, X):
        '''
        Normalize X with presetted mean and std
        '''
        X_norm = (X - self.mu)/(self.std)
        return X_norm
        
    # Fuction to append a new model
    def append(self, name, **kwargs):
        '''
        Add Models to the list of individual models.
        '''
        var = emodel(name, **kwargs)
        if(var.em != None):
            self.models.append(var)
        else:
            pass
    
    # PCA applied for feature reduction
    def reduce_feature(self, X, tol = 0.99):
        '''
        Feature reduction PCA
        
        X   : N x D shape data matrix
        tol : tolerance over data information to be kept
        '''
        for i in range(X.shape[1]):
            self.fr = PCA(n_components = i)
            self.fr.fit(X)
            tot = np.sum(self.fr.explained_variance_ratio_[0:i])
            if(tot >= tol):
                break;
        self.ndims = self.fr.explained_variance_ratio_.shape[0]

        X_red = self.fr.transform(X) 
        self.normalize_set(X_red)
        
    # Training all models
    def train(self, X, Y, Xval, Yval):
        '''
        Training the models
        '''
        X_red = self.fr.transform(X) 
        X_red_nor = self.normalize(X_red)
        # training each model individually
        for model in self.models:
            '''
            Training each model individually
            '''
            model.fit(X_red_nor,Y)
        
        
    # taking predictions from all the models
    def vote(self,X):
        '''
        Arbiter Models
        
        Takes votes of all the models
        '''
        votes = None
        flag = False
        for model in self.models:
            var = model.predict(X)
            if(not flag):
                votes = var.reshape(-1,1)
                flag = True
            else:
                votes = np.hstack((votes,var.reshape(-1,1)))
        
        return votes

    # To calculate accuracy
    def score(self,X,Y):
        '''
        Calculates the accuracy of ensemmble model.
        '''
        
        Yp = self.predict(X)
        cper = np.sum(Yp.reshape(-1,1) == Y.reshape(-1,1))/Y.shape[0]
        return cper
    
    # To predict class
    def predict(self, X):
        '''
        Predicts the class array of a given input array
        '''
        
        X_red = self.fr.transform(X)
        Xt = self.normalize(X_red)
        
        votes = self.vote(Xt)
        Y = np.squeeze(stats.mode(votes,axis=1)[0])
        return Y

In [ ]:
model = classifier() # Creating a new ensemble classifier

### Visualizing Extracted features

In [ ]:
model.reduce_feature(X_train,tol=0.999999)
ndims = model.fr.explained_variance_ratio_.shape[0]
X_red_nor = model.fr.transform(X_train)
print('Features retained = ', ndims)
X_red = model.fr.transform(X_train)
X_red_nor = model.normalize(X_red)
# print(X_norm.shape, Y_train.shape, X_red_nor.shape)
fig, ax = plt.subplots(ndims,ndims,figsize =[10,10])
for i in range(ndims):
    for j in range(ndims):
        ax[i][j].scatter(X_red_nor[:,i],X_red_nor[:,j],c = Y_train, cmap="jet", marker ='.', alpha = 0.1)
plt.savefig('scatterplot.png')
plt.show()

In [ ]:
lams = model.fr.explained_variance_ratio_
sums = [np.sum(lams[0:i]) for i in range(lams.shape[0]+1)]
plt.plot([1,2,3,4,5],sums)
plt.plot([1,lams.shape[0]+1],[1,1],'b--')
plt.xlabel("Numbers of retained eigen vectors", fontsize="14")
plt.ylabel("Data Information kept", fontsize="14")
plt.savefig("003.png")
plt.show()

#### You can just add and remove any number of models you like but they all should follow similar fit and predict function implementation in sklearn.

Example:

You want only one ensemmble Logistic regression then comment out everything else and keep logistic regression

In [ ]:
'''
Adding the individual models to the ensemble model.

we tried different coombinations thats why some models are commented out
'''
model.models = [] # lsit of models (empty intially)
model.append('LGR', random_state=1) # logistic regression
model.append('RFC', random_state=1) # random forest
# model.append('KNC') # K nearest neighbours
# model.append('SVM', kernel = 'rbf', random_state=1) # Support vector radial basis function kernel
# model.append('SVM', kernel = 'linear', random_state=1) # support vector linear kernel
# model.append('SVM', kernel = 'sigmoid', random_state=1) # support vector linear kernel
model.append('SVM', kernel = 'poly', degree = 5, random_state=1) # support vecotr polynomial kernel degree = 3
model.append('SVM', kernel = 'poly', degree = 8, random_state=1) # support vecotr polynomial kernel degree = 3
# model.append('SVM', kernel = 'poly', degree = 12, random_state=1) # support vecotr polynomial kernel degree = 3
#### MLP mean muti layer perceptron (Artificial Nueral Networks) ###
model.append('MLP', solver='lbfgs', alpha=1, hidden_layer_sizes=(25,25,2), max_iter = 10000, random_state = 12)
model.append('MLP', solver='lbfgs', alpha=1, hidden_layer_sizes=(5,5,2), max_iter = 10000, random_state = 10)
model.append('MLP', solver='lbfgs', alpha=2, hidden_layer_sizes=(10,2), max_iter = 10000, random_state = 1)

In [ ]:
model.train(X_train, Y_train, X_val, Y_val) # training the ensembel model

In [ ]:
'''
Calculating the Accuracies
'''
var_test = model.score(X_test,Y_test)
var_val = model.score(X_val,Y_val)
var_train = model.score(X_train,Y_train)
print('Test Accuracy:',var_test,'\nVal Accuracy:',var_val,'\nTrain Accuracy:',var_train,)

### Finding the confusion matrix

In [ ]:
Y_preds_test = model.predict(X_test)
Y_preds_train = model.predict(X_train)
Y_preds_val = model.predict(X_val)

In [ ]:
print("Test data confusion matrix\n--------------------------------")
print(confusion_matrix(Y_test, Y_preds_test))

In [ ]:
print("Train data confusion matrix\n--------------------------------")
print(confusion_matrix(Y_train, Y_preds_train))

In [ ]:
print("Validation data confusion matrix\n--------------------------------")
print(confusion_matrix(Y_val, Y_preds_val))

#### Observations

FP_train = 0, FN_train = 0

FP_test = 1, FN_test = 2

FP_val = 1, FN_val = 2

Note these values may differ on different train test splits but wont differ alot with respect to data size

### Thankyou 